# Football Parsing Code

In [32]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))


## Python Version
import sys
print("Python: {0}".format(sys.version))

from os.path import join
from fsUtils import mkSubDir, setFile, isFile, removeFile, isDir
from ioUtils import getFile, saveFile
from timeUtils import clock, elapsed
from fileUtils import getBaseFilename, getBasename, getDirname
from webUtils import getWebData, getHTML
from timeUtils import printDateTime, getDateTime, addMonths
from searchUtils import findExt
from time import sleep
from random import random
import sys
import re
from datetime import timedelta
from collections import Counter

from espngames import output, espn
from espngames import game, team, season
from espngames import historical

import datetime as dt
start = dt.datetime.now()

print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:07:29) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-04-09 21:07:43.828038


# Order To Download ESPN Data

In [33]:
hist = historical()
#hist.downloadTeamStandings()
#hist.parseAndDownloadTeamYearlyStandings()
#hist.parseTeamYearlyStandings()
#hist.downloadGameData()
#hist.parseGameData()

# Extract Game Data

# Penalty

In [34]:
from penalty import penalty

# Play Start

Current Time is Tue Apr 09, 2019 21:07:46 for Last Run


# Class Imports

In [105]:
%load_ext autoreload
%autoreload

from summary import playclass
from playYards import playyards
from playResult import playtextclass
from possession import possessionclass
from footballPlays import *
from debug import debugclass
from changePossession import possessionchangeclass
from teamStatistics import teamstatisticsclass
from gameStatistics import teamplayers, gameplayers

_ = clock("Last Run")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Wed Apr 10, 2019 00:28:24 for Last Run


# Play By Play Analysis

In [106]:
class playbyplay(espn, output):
    def __init__(self):
        self.name = "playbyplay"
        espn.__init__(self)
        output.__init__(self)
        
        self.hist = None
        self.ptc  = playtextclass()
        
    def setHistorical(self, hist):
        self.hist = hist
    
    
    def parsePlayStart(self, playStartText, playNo, numPlays, playData, debug=False):
        if len(playStartText) == 0:
            return None
        
        ps = None
        
        down = "(st|nd|rd|th)"
        num  = "(\d+)"
        word = "(\w+)"
        goal = "(GOAL|goal|Goal)"
        prep = "(on|at)"
        amp  = "(&|and)"
        
        ## Look for match of type [1st and 10 at CAL 19]
        m = re.search(r"{0}{1}\s{2}\s{3}\s{4}\s{5}\s{6}".format(num, down, amp, num, prep, word, num), playStartText)
        if m is not None:
            groups = m.groups()
            down   = groups[0]
            togo   = groups[3]
            startY = groups[6]
            side   = groups[5]
            ps     = playstartclass(down=down, togo=togo, startY=startY, side=side)
            

        ## Look for match of type [1st and GOAL at CAL 1]
        m = re.search(r"{0}{1}\s{2}\s{3}\s{4}\s{5}\s{6}".format(num, down, amp, goal, prep, word, num), playStartText)
        if m is not None:
            groups = m.groups()
            down   = groups[0]
            togo   = groups[6] ## replace goal with distance to goal
            startY = groups[6]
            side   = groups[5]
            ps     = playstartclass(down=down, togo=togo, startY=startY, side=side)
            

        ## Look for very near match without side of field [1st and 10 at 50]
        m = re.search(r"{0}{1}\s{2}\s{3}\s{4}\s{5}".format(num, down, amp, goal, prep, num), playStartText)
        if m is not None:
            groups = m.groups()
            down   = groups[0]
            togo   = groups[3] ## replace goal with distance to goal
            startY = groups[5]
            side   = None
            ps     = playstartclass(down=down, togo=togo, startY=startY, side=side)
            

        return ps
        raise ValueError("Could not parse [{0}] and [{1}]".format(playStartText, playData))
        
        
        
    
    def parseClockAndQuarter(self, playData, debug=False):
        gameclock = None
        quarterNo = None
        
        ## Check for end of quarter/half/game
        if sum([x in playData for x in ["End of the ", "End Of The ", "end of the ", "end of ", "End of "]]) > 0:
            gameclock = timedelta(minutes=0, seconds=0) 
            if sum([x in playData for x in ["1st Quarter", "1st quarter"]]) > 0:
                quarterNo = 1
            elif sum([x in playData for x in ["2nd Quarter", "2nd quarter", "1st Half", "1st half", "half", "Half"]]) > 0:
                quarterNo = 2
            elif sum([x in playData for x in ["3rd Quarter", "3rd quarter"]]) > 0:
                quarterNo = 3
            elif sum([x in playData for x in ["4th Quarter", "4th quarter", "2nd Half", "2nd half", " Game", " game"]]) > 0:
                quarterNo = 4
            elif "OT" in playData:
                quarterNo = "OT"
            else:
                quarterNo = "UNKNOWN END QTR"
                #raise ValueError("Could not determine quarter from {0}".format(playData))

                
        if gameclock is None and quarterNo is None:
            ## Check for start of quarter/half/game
            if sum([x in playData for x in ["Start of the ", "Start Of The ", "start of the ", "start of "]]) > 0:
                gameclock = timedelta(minutes=15, seconds=0) 
                if sum([x in playData for x in ["1st Quarter", "1st quarter"]]) > 0:
                    quarterNo = 1
                elif sum([x in playData for x in ["2nd Quarter", "2nd quarter", "1st Half", "1st half"]]) > 0:
                    quarterNo = 2
                elif sum([x in playData for x in ["3rd Quarter", "3rd quarter", "2nd Half", "2nd half"]]) > 0:
                    quarterNo = 3
                elif sum([x in playData for x in ["4th Quarter", "4th quarter", "2nd Half", "2nd half", " Game", " game"]]) > 0:
                    quarterNo = 4
                elif "OT" in playData:
                    quarterNo = "OT"
                else:
                    quarterNo = "UNKNOWN START QTR"
                    #raise ValueError("Could not determine quarter from {0}".format(playData))

        
        if gameclock is None and quarterNo is None:

            ## Check for specific format (HH:MM - 1st)
            m = re.match(r"\((.*?)\)", playData)
            if m is None:
                raise ValueError("Could not determine time and quarter from {0}".format(playData))

            try:
                text = m.group(0)
            except:
                raise ValueError("Could not extract regex result from {0}".format(playData))

               
            ## Check for overtime
            try:
                result = text[1:-1]
                if "OT" in result:
                    gameclock = timedelta(minutes=0, seconds=0) 
                    quarterNo = "OT"
            except:
                pass
            

            if gameclock is None:
                try:
                    result = text[1:-1]
                    result = result.split(" - ")
                    gameclock,quarter = result
                except:
                    raise ValueError("Could not parse time data: {0}".format(text))


                try:
                    minutes,seconds = gameclock.split(":")
                    gameclock = timedelta(minutes=int(minutes), seconds=int(seconds)) 
                except:
                    raise ValueError("Could not create timedelta for {0}".format(gameclock))


                quarterNo = {"1st":1, "2nd": 2, "3rd": 3, "4th": 4}.get(quarter)
                if quarterNo is None:
                    print("Could not find quarter from [{0}] or [{1}]".format(quarter, text))
                    #raise ValueError("Could not find quarter from [{0}] or [{1}]".format(quarter, text))


                try:
                    playData = playData.replace(text, "").strip()
                except:
                    raise ValueError("Could not remove [{0}] from [{1}]".format(text, playData))
                
        
        return gameclock,quarterNo,playData
    
    
    
    ########################################################################################################
    ##
    ## Combine all play data data together
    ##
    ########################################################################################################
    def parsePlayData(self, playStart, playData, fieldMap, possData, debug=False):
        if debug:
            fname = sys._getframe().f_code.co_name
            print("FUNC {0}".format(fname))
        
        
        ##########################
        ## Find clock and quarter
        ##########################
        gameclock, quarter, playText = self.parseClockAndQuarter(playData, debug=debug)
        
        
        ##########################
        ## Determine play type
        ##########################
        ptype = playtype(playText)
        play  = ptype.getPlay()
        if ptype.valid is True:
            if play is None:
                raise ValueError("Could not determine play type from {0}".format(playText))
            play.analyze(debug=debug)
        if debug:
            print("\tPlay: {0} \t{1}".format(play.name, play.text))

        
        ##########################
        ## Check for valid play
        ##########################
        if ptype.valid is True and play is not None:
            validPlay = play.pa.getKey("noplay") != True
            if isinstance(play, (beginplay, endplay, timeoutplay)):
                validPlay = False
        else:
            validPlay = False
        
        
        
        ##########################
        ## Determine possession
        ##########################        
        if ptype.valid is True and play is not None:
            playPossession = possData.determinePossession(play, debug=debug)
        else:
            playPossession = playpossessionclass(start=None, end=None)
            validPlay = False

        
        if playPossession.valid is False:
            validPlay = False
            
        
        ##########################
        ## Insert Clock/Quarter
        ##########################
        if playStart is None:
            playStart = playstartclass()
            
        if isinstance(playStart, playstartclass):
            playStart.setQuarter(quarter)
            playStart.setGameClock(gameclock)

            
        ### Result of play
        playResult = playclass(possession=playPossession, start=playStart, play=play, valid=validPlay)
        return playResult

        
    def parsePossession(self, possession, fieldMap, debug=False):
        if debug:
            fname = sys._getframe().f_code.co_name
            print("FUNC {0}".format(fname))
            
        if not isinstance(possession, list):
            raise ValueError("Possession is not a list: {0}".format(possession))
            
        if len(possession) != 1:
            raise ValueError("Not one element in possession list: {0}".format(possession))
            
        teamID = possession[0]
        
        teamAbbrev = None
        try:
            teamAbbrev = fieldMap[teamID]
        except:
            raise ValueError("Could not find {0} in field map: {1}".format(teamID, fieldMap))
            
        return teamAbbrev
        
        
    def augmentPlayStartWithPossession(self, possession, startVals, debug=False):
        if debug:
            fname = sys._getframe().f_code.co_name
            print("FUNC {0}".format(fname))
            
        if isinstance(startVals, playstartclass):
            
            ## Check for a team having the ball on their own side of the field (common)
            try:
                startY = int(startVals.startY)
            except:
                raise ValueError("Starting line {0} is not an integer".format(startVals.startY))

            if possession == startVals.side:
                distToEndZone = 100 - startY
            else:
                distToEndZone = startY                
                
            startVals.setStartY(startY)
            startVals.setDistToEndZone(distToEndZone)

        return startVals
    

    def parseHeadline(self, headline, debug=False):
        if debug:
            fname = sys._getframe().f_code.co_name
            print("FUNC {0}".format(fname))
            
        pltype = None
        
        if isinstance(headline, list):
            if len(headline) >= 1:
                ptype  = playtype(headline[0])
                pltype = ptype.getPlay()
                
                if pltype is None:
                    raise ValueError("Could not determine headline play type: {0}".format(headline[0]))
            else:
                print("Not one headline entry: {0}".format(headline))
        else:
            raise ValueError("Headline is not a list: {0}".format(headline))
            
        return pltype
    
    
    def parseScore(self, score, debug=False):
        if debug:
            fname = sys._getframe().f_code.co_name
            print("FUNC {0}".format(fname))
            
        if not isinstance(score, list):
            raise ValueError("Could not determine score type: {0}".format(score))
            
        if len(score) != 1:
            raise ValueError("Not one detail entry: {0}".format(score))
            
        scoredata = score[0]
        
        try:
            scoredata = int(scoredata)
        except:
            raise ValueError("Could not find an integer score for {0}".format(scoredata))
            
        return scoredata
    
    
    def parseDetail(self, detail, debug=False):
        if debug:
            fname = sys._getframe().f_code.co_name
            print("FUNC {0}".format(fname))
            
        if not isinstance(detail, list):
            raise ValueError("Could not determine detail play type: {0}".format(detail))
            
        if len(detail) != 1:
            raise ValueError("Not one detail entry: {0}".format(detail))
            
        detaildata = detail[0]
        
        
        yards = "(yards|yard|Yds|yds|Yd|yd)"
        plays = "(play|plays)"        
        num  = "([+-?]\d+|\d+)"
        
        totalplays = None
        totalyards = None
        totalclock = None

        m = re.search(r"{0}\s{1},\s{2}\s{3},\s{4}:{5}".format(num, plays, num, yards, num, num), detaildata)
        if m is not None:
            groups = m.groups()
            totalplays = int(groups[0])
            totalyards = int(groups[2])
            totalclock = timedelta(minutes=int(groups[4]), seconds=int(groups[5]))

            
        if totalplays is None and totalyards is None and totalclock is None:
            m = re.search(r"{0}\s{1},\s{2}\s{3}".format(num, plays, num, yards), detaildata)
            if m is not None:
                groups = m.groups()
                totalplays = int(groups[0])
                totalyards = int(groups[2])
                totalclock = timedelta(minutes=0, seconds=0)
                
            
        if totalplays is None and totalyards is None and totalclock is None:
            raise ValueError("Could not parse drive detail: {0}".format(detaildata))

        drivedetail = drivedetailclass(plays=totalplays, yards=totalyards, gametime=totalclock)
            
        return drivedetail
    
    
    
        
        
        
    
    ########################################################################################################
    ##
    ## Analyze Possession For Every Play
    ##
    ########################################################################################################
    def analyzePossession(self, gameData, players, debug=False):
        if debug:
            fname = sys._getframe().f_code.co_name
            print("FUNC {0}".format(fname))
            
        if not isinstance(gameData, list):
            raise ValueError("Game data is not a list!")
            
        unknown = False

        for idr,driveData in enumerate(gameData):
            drivePlays = driveData.plays
            prevStart  = None
            for ipl,drivePlay in enumerate(drivePlays):
                play       = drivePlay.play
                valid      = drivePlay.valid
                if valid is False:
                    continue
                possession = drivePlay.possession
                text       = possession.text
                status     = ""
                if possession.isUnknownStart():
                    unknown = True
                    status  = "<---- UNKNOWN"
                if possession.isPreviousStart():
                    if ipl == 0:
                        unknown = True
                        status  = "<---- UNKNOWN"
                    else:
                        try:
                            nextStart = drivePlays[ipl+1].possession.start
                        except:
                            nextStart = None
                        
                        if nextStart == prevStart:
                            possession.start = prevStart
                            status  = "<---- PREV/NXT"
                        else:
                            status  = "<---- PREV/UNK"
                prevStart = possession.start
                keys = ",".join([k for k,v in play.pa.__dict__.items() if v is True])
                if status == "":
                    status = keys
                if debug:
                    print("{0}\t{1}\t{2}\t{3: <15}{4: <15}{5: <5}{6: <40}{7}".format(idr,ipl,str(possession.start),str(play.name),str(possession.player),str(possession.position),str(status),str(text)))
            if debug:
                print("")

        if unknown is True:
            if debug:
                print("UNKNOWN PLAY START IN GAME")
            
        return gameData
        
    
    ########################################################################################################
    ##
    ## Analyze Kickoff Structure
    ##
    ########################################################################################################
    def analyzeKickoffs(self, gameData, players, copMap, debug=False):
        if debug:
            fname = sys._getframe().f_code.co_name
            print("FUNC {0}".format(fname))
            
        if not isinstance(gameData, list):
            raise ValueError("Game data is not a list!")
                        
        unknown = False
        
        kickers = {}
        
        for idr,driveData in enumerate(gameData):
            drivePlays = driveData.plays
            prevStart  = None
            if sum([isinstance(x.play, kickoffplay) for x in drivePlays]) > 0:
                posCntr = Counter()
                for playData in drivePlays:
                    if isinstance(playData.play, (rushingplay, passingplay, sackplay, fieldgoalplay)):
                        posCntr[playData.possession.start] += 1
                        
                if len(posCntr) == 0:
                    continue
                
                for ipl,playData in enumerate(drivePlays):
                    if isinstance(playData.play, (kickoffplay)):
                        playText = playData.play.text
                        keys = [" kickoff ", " Kickoff", " KICKOFF "]
                        pos = max([playText.find(x) for x in keys])
                        if pos > 0:
                            text = playText[:pos].strip()
                            if kickers.get(text) is None:
                                kickers[text] = Counter()
                            try:
                                team = copMap[posCntr.most_common(1)[0][0]]
                            except:
                                if debug:
                                    print("Unknown team: {0} in {1}".format(team, fname))
                                continue
                                
                            try:
                                kickers[text][team] += 1
                            except:
                                pass
                            
                        
        for kicker in kickers.keys():
            kickers[kicker] = kickers[kicker].most_common(1)[0][0]
            if debug:
                print("Kicker: {0} ---> {1}".format(kicker,kickers[kicker]))
        players.addKickers(kickers, debug)
        
        
        for idr,driveData in enumerate(gameData):
            for ipl,playData in enumerate(driveData.plays):
                if isinstance(playData.play, (kickoffplay)):
                    poss, self.determinePossession(playData.play, players)
                    gameData[idr].plays[ipl].possession = poss

        gameData = self.analyzePossession(gameData, players, debug)
            
        return gameData
        
    
    ########################################################################################################
    ##
    ## Analyze Interception Structure
    ##
    ########################################################################################################
    def analyzeInterceptions(self, gameData, players, debug=False):
        if debug:
            fname = sys._getframe().f_code.co_name
            print("FUNC {0}".format(fname))
            
        if not isinstance(gameData, list):
            raise ValueError("Game data is not a list!")
        
        for idr,driveData in enumerate(gameData):
            drivePlays = driveData.plays
            for ipl,playData in enumerate(drivePlays):
                if playData.play.pa.getKey("interception") is True:                    
                    if playData.possession.isPreviousStart() is True or playData.possession.isUnknownStart() is True:
                        try:
                            if gameData[idr].plays[ipl-1].valid is True:
                                gameData[idr].plays[ipl].possession.start = gameData[idr].plays[ipl-1].possession.start
                        except:
                            pass
                        
        return gameData
        
    
    ########################################################################################################
    ##
    ## Analyze Penalties
    ##
    ########################################################################################################
    def analyzePenalties(self, gameData, players, debug=False):
        if debug:
            fname = sys._getframe().f_code.co_name
            print("FUNC {0}".format(fname))
            
        if not isinstance(gameData, list):
            raise ValueError("Game data is not a list!")
        
        for idr,driveData in enumerate(gameData):
            for ipl,playData in enumerate(driveData.plays):
                if isinstance(playData.play, penaltyplay):
                    if playData.possession.isPreviousStart() is True or playData.possession.isUnknownStart() is True:
                        try:
                            if gameData[idr].plays[ipl-1].possession.start == gameData[idr].plays[ipl+1].possession.start:
                                gameData[idr].plays[ipl].possession.start = gameData[idr].plays[ipl-1].possession.start
                        except:
                            pass
                    
                    if playData.possession.isPreviousStart() is True or playData.possession.isUnknownStart() is True:

                        try:
                            if gameData[idr].plays[ipl+1].possession.isPreviousStart() is False and gameData[idr].plays[ipl+1].valid is True:
                                gameData[idr].plays[ipl].possession.start = gameData[idr].plays[ipl+1].possession.start
                        except:
                            pass
                    
                    if playData.possession.isPreviousStart() is True or playData.possession.isUnknownStart() is True:
                        try:
                            if gameData[idr].plays[ipl-1].valid is True:
                                gameData[idr].plays[ipl].possession.start = gameData[idr].plays[ipl-1].possession.start
                        except:
                            pass



        gameData = self.analyzePossession(gameData, players, debug)
                        
        return gameData

    
    
    
    ########################################################################################################
    ##
    ## Analyze Game Score After Each Drive
    ##
    ########################################################################################################
    def analyzeGameScore(self, gameData, players, homeTeamGameData, awayTeamGameData, debug=False):
        if debug:
            fname = sys._getframe().f_code.co_name
            print("FUNC {0}".format(fname))
            
        if not isinstance(gameData, list):
            raise ValueError("Game data is {0}".format(type(gameData)))

        homeTeam = players.homeTeamName
        awayTeam = players.awayTeamName

        finalHomeScore = None
        runningHomeScore = 0
        finalAwayScore = None
        runningAwayScore = 0
        if debug:
            print("{0: <7}{1: <4}{2: <4}{3: <4}{4: <4}\t(Post Drive Score)".format("Drive", "H", "(H)", "A", "(A)"))
            
        for idr in range(len(gameData)):
            driveData  = gameData[idr]
            homescore  = driveData.postdrivehomescore
            awayscore  = driveData.postdriveawayscore
            finalAwayScore = awayscore
            finalHomeScore = homescore

            drivePlays = driveData.plays
            if isinstance(drivePlays, list) is False:
                raise ValueError("Drive Plays is not a list object!")

            scoringplays = []
            for drivePlay in drivePlays:
                play       = drivePlay.play
                possession = drivePlay.possession
                score      = play.pa.isScore()
                if score > 0:
                    scoringplays.append(play.text)
                    if possession.start == homeTeam:
                        runningHomeScore += score
                    elif possession.start == awayTeam:
                        runningAwayScore += score
                    else:
                        print("Possession {0} is neither home {1} or away {2}".format(possession.start, homeTeam, awayTeam))
                        
            if debug:
                print("{0: <7}{1: <4}{2: <4}{3: <4}{4: <4}{5}".format(idr, runningHomeScore, homescore, runningAwayScore, awayscore, scoringplays))
                
                
        if debug:
            print("{0: <7}{1: <4}{2: <4}{3: <4}{4: <4}{5}".format("----", "---", "---", "---", "---", ""))
            print("{0: <7}{1: <4}{2: <4}{3: <4}{4: <4}{5}".format("END", runningHomeScore, finalHomeScore, runningAwayScore, finalAwayScore, ""))
            print("{0: <7}{1: <4}{2: <4}{3: <4}{4: <4}{5}".format("TRUE", homeTeamGameData.teamAScore, "", awayTeamGameData.teamAScore, "", ""))
        
        diffHomeScore = homeTeamGameData.teamAScore - runningHomeScore
        diffAwayScore = awayTeamGameData.teamAScore - runningAwayScore

        if debug:
            print("{0: <7}{1: <4}{2: <4}{3: <4}{4: <4}{5}".format("DIFF", diffHomeScore, "", diffAwayScore, "", ""))
            
        if diffHomeScore != 0 or diffAwayScore != 0:
            return False
        return True
    
    
                        
    ######################################################
    ## Save Augmented Stats Data
    ######################################################
    def saveAugmentedStatsData(self, players, homeTeamMetaData, awayTeamMetaData, debug=False):
        if debug:
            fname = sys._getframe().f_code.co_name
            print("FUNC {0}".format(fname))
            
        augmentedStatsFilename = setFile(self.hist.getStatisticsResultsDir(), "{0}-stats-extra.json".format(year))
        if isFile(augmentedStatsFilename) and False:
            augmentedStatsData = getFile(augmentedStatsFilename)
        else:
            augmentedStatsData = {}

        if augmentedStatsData.get(homeTeamMetaData["ID"]) is None:
            augmentedStatsData[homeTeamMetaData["ID"]] = {}
        if augmentedStatsData[homeTeamMetaData["ID"]].get('Kickers') is None:
            augmentedStatsData[homeTeamMetaData["ID"]]['Kickers'] = {}
        if augmentedStatsData[homeTeamMetaData["ID"]].get('Punters') is None:
            augmentedStatsData[homeTeamMetaData["ID"]]['Punters'] = {}

        for name in players.homeTeamPlayers.kickers:
            augmentedStatsData[homeTeamMetaData["ID"]]['Kickers'][name] = "K"
        for name in players.homeTeamPlayers.punters:
            augmentedStatsData[homeTeamMetaData["ID"]]['Punters'][name] = "P"


        if augmentedStatsData.get(awayTeamMetaData["ID"]) is None:
            augmentedStatsData[awayTeamMetaData["ID"]] = {}
        if augmentedStatsData[awayTeamMetaData["ID"]].get('Kickers') is None:
            augmentedStatsData[awayTeamMetaData["ID"]]['Kickers'] = {}
        if augmentedStatsData[awayTeamMetaData["ID"]].get('Punters') is None:
            augmentedStatsData[awayTeamMetaData["ID"]]['Punters'] = {}

        for name in players.awayTeamPlayers.kickers:
            augmentedStatsData[awayTeamMetaData["ID"]]['Kickers'][name] = "K"
        for name in players.awayTeamPlayers.punters:
            augmentedStatsData[awayTeamMetaData["ID"]]['Punters'][name] = "P"

        if debug:
            print("Saving augmented stats to {0}".format(augmentedStatsFilename))
        saveFile(idata=augmentedStatsData, ifile=augmentedStatsFilename, debug=True)                        
        
    
    
    ################################################################################################################################################
    ##
    ##
    ## Main Loop To Process Games
    ##
    ##
    ################################################################################################################################################
    def parseGames(self, gameID=None, test=False, debug=False):
        if self.hist is None:
            raise ValueError("Must set historical class!")

        self.unknownPlays = []
        
        self.poorlyParsed = ['401012731', '400547781', '400547808']
        
        self.statsToGet = {}
        
        pcc = possessionchangeclass()
        dc  = debugclass()
            
        files    = findExt(self.hist.getGamesResultsDir(), ext=".p", debug=debug)
        for ifile in files:
            print(ifile)
            try:
                year = int(getBaseFilename(ifile).split("-")[0])
            except:
                raise ValueError("Could not get year from {0}".format(ifile))
            
            if year != 2014:
                continue
                
            yearData = getFile(ifile)
            
            seasonFilename = setFile(self.hist.getSeasonResultsDir(), "{0}.p".format(year))
            seasonData     = getFile(seasonFilename)
            
            statsFilename = setFile(self.hist.getStatisticsResultsDir(), "{0}-stats.json".format(year))
            statsData     = getFile(statsFilename)
            
            augmentedStatsFilename = setFile(self.hist.getStatisticsResultsDir(), "{0}-stats-extra.json".format(year))
            augmentedStatsData     = getFile(augmentedStatsFilename)
                

            if len(self.unknownPlays) > 75:
                break


            
            totalGames = 0
            for gameIdent,gameData in yearData.items():
                if gameID is not None:
                    if gameID != gameIdent:
                        continue
                
                teamsMetaData    = gameData["Teams"]
                homeTeamMetaData = teamsMetaData["Home"]
                awayTeamMetaData = teamsMetaData["Away"]
                driveData        = gameData["Plays"]
                
                
                print("GAME ID: {0}".format(gameIdent))

                fieldMap = {}
                fieldMap[homeTeamMetaData["ID"]]     = homeTeamMetaData["Abbrev"]
                fieldMap[homeTeamMetaData["Abbrev"]] = homeTeamMetaData["ID"]
                fieldMap[awayTeamMetaData["ID"]]     = awayTeamMetaData["Abbrev"]
                fieldMap[awayTeamMetaData["Abbrev"]] = awayTeamMetaData["ID"]
                
                fieldMap["Home"] = homeTeamMetaData["Abbrev"]
                fieldMap["Away"] = awayTeamMetaData["Abbrev"]
                
                copmap = {}
                copmap[homeTeamMetaData["Abbrev"]] = awayTeamMetaData["Abbrev"]
                copmap[awayTeamMetaData["Abbrev"]] = homeTeamMetaData["Abbrev"]

                
                homeTeamSeasonData = seasonData.teams.get(homeTeamMetaData["ID"])
                if homeTeamSeasonData is None:
                    continue
                    raise ValueError("There is no home team season data for {0}, {1}".format(homeTeamMetaData["Abbrev"], homeTeamMetaData["ID"]))
                homeTeamGameData = [x["Game"] for x in homeTeamSeasonData.games if x["Game"].gameID == gameIdent]
                try:
                    homeTeamGameData = homeTeamGameData[0]
                except:
                    continue
                    raise ValueError("There is no home team game data")

                awayTeamSeasonData = seasonData.teams.get(awayTeamMetaData["ID"])
                if awayTeamSeasonData is None:
                    continue
                    raise ValueError("There is no away team season data for {0}, {1}".format(awayTeamMetaData["Abbrev"], awayTeamMetaData["ID"]))
                awayTeamGameData = [x["Game"] for x in awayTeamSeasonData.games if x["Game"].gameID == gameIdent]
                try:
                    awayTeamGameData = awayTeamGameData[0]
                except:
                    continue
                    raise ValueError("There is no away team game data")

                
                


                if gameIdent in self.hist.noGameData:
                    continue
                    
                if gameIdent in self.poorlyParsed:
                    continue
                                
                
                ################################################################################
                ### Learn key plays for use in determining possession
                ################################################################################
                players = gameplayers(teamsMap=fieldMap, statsData=statsData)
                players.augmentData(augmentedStatsData)
                
                possData = possessionclass(players)
                
                if debug:
                    print("")
                    print("  Home Team: {0}".format(homeTeamMetaData))
                    print("\tPassers\t",players.homeTeamPlayers.passers)
                    print("\tRunners\t",players.homeTeamPlayers.runners)
                    print("\tKickers\t",players.homeTeamPlayers.kickers)
                    print("\tPunters\t",players.homeTeamPlayers.punters)
                    print("")
                    print("  Away Team: {0}".format(awayTeamMetaData))
                    print("\tPassers\t",players.awayTeamPlayers.passers)
                    print("\tRunners\t",players.awayTeamPlayers.runners)
                    print("\tKickers\t",players.awayTeamPlayers.kickers)
                    print("\tPunters\t",players.awayTeamPlayers.punters)
                    print("")


                
                if test:
                    saveFile(idata=driveData, ifile="testDriveData.p", debug=True)
                    saveFile(idata=fieldMap,  ifile="testFieldMap.p",  debug=True)
                    saveFile(idata=statsData,  ifile="testGameStats.p", debug=True)
                    saveFile(idata=augmentedStatsData,  ifile="testAugStats.p", debug=True)
                    1/0

                if len(self.unknownPlays) > 75:
                    break
                    
                
                gameResult = []
                totalPlays = 0
                
                
                ################################################################################
                ### Iterate over drives
                ################################################################################
                for idr,drive in enumerate(driveData):
                    ### Fix known drive problems
                    #drive = self.fixDrive(gameIdent, idr, drive)
                    
                    driveNo    = drive['Drive']
                    headline   = drive['Headline']
                    detail     = drive['Detail']
                    ### For whatever reason home/away scores are reversed on the webpage...
                    homescore  = drive['AwayScore']
                    awayscore  = drive['HomeScore']
                    possession = drive['Posession'] ## note this needs to change later
                    drivedata  = drive['Data']
                    
                    
                    
                    if debug:
                        print("Drive: {0} \t{1} (H) - {2} (A)".format(idr,homescore, awayscore))
                    
    
                    ## Determine possession as defined in the play start fields later
                    possession = self.parsePossession(possession, fieldMap)
                    headline   = self.parseHeadline(headline)
                    detail     = self.parseDetail(detail)
                    homescore  = self.parseScore(homescore)
                    awayscore  = self.parseScore(awayscore)
            
        
                    fullDriveData = driveclass(headline=headline, detail=detail, possession=possession,
                                              postdrivehomescore=homescore, postdriveawayscore=awayscore)
                    
                    drivePlays = []
            
                
                    ################################################################################
                    ### Iterate over plays in drive
                    ################################################################################
                    for ipl,play in enumerate(drivedata):
                        playNo        = play['Play']
                        playStartText = play['Start']
                        playData      = play['Data']

                        ### Determine play starting position and clock
                        startVals = self.parsePlayStart(playStartText, playNo, len(drivedata), playData, debug=debug)
                        startVals = self.augmentPlayStartWithPossession(possession, startVals, debug=debug)
                        
                        ### Determine type of play
                        playResult = self.parsePlayData(startVals, playData, fieldMap, possData, debug)
                        
                        
                        if playResult.possession.start is None and playResult.valid is True:
                            raise ValueError("ERROR WITH POSSESSION:",playResult.play.name,'\t',playResult.play.text)
                        
                        totalPlays += 1
                        drivePlays.append(playResult)

                    
                    fullDriveData.setPlays(drivePlays)
                    #fullDriveData = self.insertMissingData(gameIdent, idr, fullDriveData, debug)
                    #fullDriveData = self.augmentPlayWithScore(fullDriveData, fieldMap, debug)
                    gameResult.append(fullDriveData)


                
                
                ################################################################################
                ### Analyze Possession
                ################################################################################ 
                dc.showGame(gameResult, debug=debug)
                gameResult = self.analyzePossession(gameResult, players, debug=debug)
                #gameResult = self.analyzeKickoffs(gameResult, players, copmap, debug=debug)
                #gameResult = self.analyzeInterceptions(gameResult, players, debug=debug)
                gameResult = pcc.splitChangeOfPossession(gameResult, players, copmap, debug=debug)
                gameResult = self.analyzePenalties(gameResult, players, debug=debug)
                dc.showGame(gameResult, debug=debug)
                
                scoreResult = self.analyzeGameScore(gameResult, players, homeTeamGameData, awayTeamGameData, debug=True)
                if scoreResult is False:
                    dc.showGame(gameResult, debug=True)
                    
                    saveFile(idata=driveData, ifile="testDriveData.p", debug=True)
                    saveFile(idata=fieldMap,  ifile="testFieldMap.p",  debug=True)
                    saveFile(idata=statsData,  ifile="testGameStats.p", debug=True)
                    saveFile(idata=augmentedStatsData,  ifile="testAugStats.p", debug=True)
                    
                    raise ValueError("There was a problem parsing the scores for this game {0}!".format(gameIdent))

                #self.saveAugmentedStatsData(players, homeTeamMetaData, awayTeamMetaData, debug)
                
                
                #self.analyzeGameData(gameResult)


                    
                totalGames += 1
                if debug:
                    print("Found {0} plays in this game {1}".format(totalPlays, gameID))
                    
            print("Found {0} total games for {1}".format(totalGames, ifile))
                
_, _ = clock("Last Run")

Current Time is Wed Apr 10, 2019 00:28:25 for Last Run


In [107]:
#tsc = teamstatisticsclass()
#tsc.collect(hist, test=False, debug=True)

In [108]:
pbp = playbyplay()
pbp.setHistorical(hist)
pbp.parseGames(gameID='400547648', test=False, debug=False)

/Users/tgadfort/Documents/football/results/games/2009-games.p
/Users/tgadfort/Documents/football/results/games/2010-games.p
/Users/tgadfort/Documents/football/results/games/2011-games.p
/Users/tgadfort/Documents/football/results/games/2008-games.p
/Users/tgadfort/Documents/football/results/games/2018-games.p
/Users/tgadfort/Documents/football/results/games/2003-games.p
/Users/tgadfort/Documents/football/results/games/2013-games.p
/Users/tgadfort/Documents/football/results/games/2012-games.p
/Users/tgadfort/Documents/football/results/games/2007-games.p
/Users/tgadfort/Documents/football/results/games/2017-games.p
/Users/tgadfort/Documents/football/results/games/2016-games.p
/Users/tgadfort/Documents/football/results/games/2006-games.p
/Users/tgadfort/Documents/football/results/games/2004-games.p
/Users/tgadfort/Documents/football/results/games/2014-games.p
GAME ID: 400547648
FUNC analyzeGameScore
Drive  H   (H) A   (A) 	(Post Drive Score)
0      0   0   0   0   []
1      3   3   0   0  

In [ ]:
pbp.statsToGet

# Test Area

In [ ]:
year = 2014
driveData = getFile(ifile="testDriveData.p", debug=True)
fieldMap  = getFile(ifile="testFieldMap.p",  debug=True)
statsData = getFile(ifile="testGameStats.p",  debug=True)
augmentedStatsData = getFile(ifile="testAugStats.p",  debug=True)

players = gameplayers(teamsMap=fieldMap, statsData=statsData)
players.augmentData(augmentedStatsData)

text = "Hayden Lekacz kickoff for 64 yds , Khalif Herbin return for 18 yds to the Temp 19"
x = players.getKickingTeam(text, debug=True)
print(x)
text = "Alex Starzyk punt for 40 yds , Darrius Sims returns for 6 yds to the Vandy 44 VANDERBILT Penalty, Illegal Block (10 Yards) to the Vandy 34"
x = players.getKickingTeam(text, debug=True)
print(x)
x = players.getPuntingTeam(text, debug=True)
print(x)

fieldMap

In [ ]:
pbp = playbyplay()
pbp.setHistorical(hist)

dc = debugclass()

################################################################################
### Iterate over drives
################################################################################
debug=True
for idr,drive in enumerate(driveData):
    drivedata  = drive['Data']
    
    ################################################################################
    ### Iterate over drives
    ################################################################################
    for idr,drive in enumerate(driveData):
        ### Fix known drive problems
        #drive = self.fixDrive(gameIdent, idr, drive)

        driveNo    = drive['Drive']
        headline   = drive['Headline']
        detail     = drive['Detail']
        ### For whatever reason home/away scores are reversed on the webpage...
        homescore  = drive['AwayScore']
        awayscore  = drive['HomeScore']
        possession = drive['Posession'] ## note this needs to change later
        drivedata  = drive['Data']

                    
        ## Determine possession as defined in the play start fields later
        possession = pbp.parsePossession(possession, fieldMap)
        headline   = pbp.parseHeadline(headline)
        detail     = pbp.parseDetail(detail)
        homescore  = pbp.parseScore(homescore)
        awayscore  = pbp.parseScore(awayscore)
            
        
        fullDriveData = driveclass(headline=headline, detail=detail, possession=possession,
                                  postdrivehomescore=homescore, postdriveawayscore=awayscore)

        drivePlays = []
            
                
        ################################################################################
        ### Iterate over plays in drive
        ################################################################################
        for ipl,play in enumerate(drivedata):
            playNo        = play['Play']
            playStartText = play['Start']
            playData      = play['Data']

            ### Determine play starting position and clock
            startVals = pbp.parsePlayStart(playStartText, playNo, len(drivedata), playData, debug=debug)
            startVals = pbp.augmentPlayStartWithPossession(possession, startVals, debug=debug)

            ### Determine type of play
            playResult = pbp.parsePlayData(startVals, playData, fieldMap, players, debug)
            
            dc.possession(playResult, driveNo=idr, playNo=ipl, debug=True)

In [ ]:
text = "Hayden Lekacz kickoff for 64 yds , Khalif Herbin return for 18 yds to the Temp 19"
x = players.getKickingTeam(text, debug=True)
print(x)
text = "Alex Starzyk punt for 40 yds , Darrius Sims returns for 6 yds to the Vandy 44 VANDERBILT Penalty, Illegal Block (10 Yards) to the Vandy 34"
x = players.getKickingTeam(text, debug=True)
print(x)
x = players.getPuntingTeam(text, debug=True)
print(x)

In [ ]:
################################################################################
### Learn key plays for use in determining possession
################################################################################
gp = gameplayers(teamsMap=fieldMap, statsData=statsData)
print(gp.getPassingTeam("Phillip Walker passes for 2 yards"))
print(gp.getPassingTeam("P.J. Walker passes for 2 yards", debug=False))
print(gp.getRunningTeam("Ralph Webb runs for 2 yards", debug=False))
print(gp.getFGKickingTeam("Austin Jones 47 Yd FG", debug=False))
print(gp.getKickingTeam("Austin Jones Kickoff for touchback", debug=False))
print(gp.getPuntingTeam("Austin Jones punt for 35 yards", debug=False))
print(gp.getPuntingTeam("A. Jones punt for 35 yards", debug=False))
print(gp.getPuntingTeam("Alex Starzyk punt for 43 yds , Darrius Sims returns for 8 yds to the Vandy 28"))
print(gp.getReceivingTeam("Phillip Walker pass complete to Jalen Fitzpatrick for 6 yds", debug=False))
print(gp.getReceivingTeam("Johnny McCrary pass complete to C.J. Duncan for 6 yds", debug=False))
print(gp.getReceivingTeam("Austin Jones pass complete to Jalen Fitzpatrick for 6 yds", debug=False))
print(gp.getPassingTeam("Austin Jones pass complete to Jalen Fitzpatrick for 6 yds", debug=False))
print(gp.homeTeamPlayers.punters)
print(gp.awayTeamPlayers.punters)

In [ ]:
from searchUtils import findNearest
x1 = ['Johnny McCrary', 'Patton Robinette', 'Wade Freebeck', 'Stephen Rivers', 'Josh Grady']
x2 = ['Phillip Walker', 'P. Walker', 'Connor Reilly', 'Jalen Fitzpatrick', 'Tim DiGiorgio']
x  = "P.J. Walker"
print(findNearest(cutoff=0.7, ilist=x1, item=x, num=1))
print(findNearest(cutoff=0.9, ilist=x2, item=x, num=1))

In [ ]:
x = ['Joe', 'Red', 'Blue']
start = ["{0}.".format(y[0]) for y in x[:-1]]
end   = x[-1]
" ".join([" ".join(start), end])

In [ ]:
kick = ("(fg|FG)")
num  = "([+-?]\d+|\d+)"  
dist = ("(yards|yard|Yds|yds|Yd|yd)")
        
x = 'Matt Severino 47 yd FG MISSED'
m = re.split("{0}\s{1}\s{2}".format(num, dist, kick), x)
if len(m) > 1:
    fgkicker = m[0].strip()
    print(fgkicker)

In [ ]:
#x = keyplayers()
txt  = "Evan O'Hara kickoff for 58 yds"
kickoff = [" kickoff ", " Kickoff ", " KICKOFF "]
pos = max([txt.find(x) for x in kickoff])
print(txt[:pos].strip())

txt = "Damien Harris run for 3 yds to the Alab 38"
kickoff = [" run ", " Run ", " RUN "]
pos = max([txt.find(x) for x in kickoff])
print(txt[:pos].strip())

txt = "Tua Tagovailoa pass complete to Damien Harris for 14 yds to the Alab 46 for a 1ST down"
kickoff = [" pass ", " Pass ", " PASS "]
pos = max([txt.find(x) for x in kickoff])
print(txt[:pos].strip())

txt = "Tua Tagovailoa sacked by G.G. Robinson for a loss of 6 yards to the Alab 32"
kickoff = [" sacked by ", " Sacked By ", " SACKED BY "]
pos = max([txt.find(x) for x in kickoff])
print(txt[:pos].strip())





In [ ]:

0 	 0 	 Evan O'Hara kickoff for 58 yds
0 	 1 	 Damien Harris run for 3 yds to the Alab 38
0 	 2 	 Tua Tagovailoa sacked by G.G. Robinson for a loss of 6 yards to the Alab 32
0 	 3 	 Tua Tagovailoa pass complete to Damien Harris for 14 yds to the Alab 46 for a 1ST down
0 	 4 	 Tua Tagovailoa pass complete to Irv Smith Jr. for 7 yds to the Lvile 47
0 	 5 	 Tua Tagovailoa pass complete to DeVonta Smith for 28 yds to the Lvile 19 for a 1ST down
0 	 6 	 Tua Tagovailoa run for 8 yds to the Lvile 11
0 	 7 	 Tua Tagovailoa pass complete to Jerry Jeudy for 11 yds for a TD, (Austin Jones KICK)
1 	 0 	 Joseph Bulovas kickoff for 50 yds for a touchback
1 	 1 	 Jawon Pass pass complete to Micky Crum for 13 yds to the Lvile 38 for a 1ST down
1 	 2 	 Jawon Pass pass complete to Micky Crum for 5 yds to the Lvile 43
1 	 3 	 Trey Smith run for 2 yds to the Lvile 45
1 	 4 	 Jawon Pass pass incomplete to Trey Smith
1 	 5 	 Mason King punt for 35 yds, fair catch by Jaylen Waddle at the Alab 20
2 	 0 	 Josh Jacobs run for 6 yds to the Alab 26
2 	 1 	 Josh Jacobs run for 2 yds to the Alab 28
2 	 2 	 Tua Tagovailoa run for 7 yds to the Alab 35 for a 1ST down
2 	 3 	 Tua Tagovailoa pass incomplete
2 	 4 	 Tua Tagovailoa pass incomplete to Josh Jacobs
2 	 5 	 Tua Tagovailoa pass complete to DeVonta Smith for 26 yds to the Lvile 39 for a 1ST down
2 	 6 	 Najee Harris run for 4 yds to the Lvile 35
2 	 7 	 Tua Tagovailoa pass complete to DeVonta Smith for 18 yds DeVonta Smith fumbled, forced by Cornelius Sturghill, recovered by Lvile Dorian Etheridge
3 	 0 	 Jawon Pass pass incomplete to Seth Dawkins
3 	 1 	 Dae Williams run for 4 yds to the Lvile 21
3 	 2 	 LOUISVILLE Penalty, Substitution Infraction (-5 Yards) to the Lvile 16
3 	 3 	 Jawon Pass pass incomplete to Chatarius Atwell
3 	 4 	 Mason King punt for 39 yds, fair catch by Jaylen Waddle at the Alab 45
4 	 0 	 Damien Harris run for 32 yds to the Lvile 23 for a 1ST down
4 	 1 	 Tua Tagovailoa pass complete to Jaylen Waddle for 6 yds to the Lvile 17
4 	 2 	 Damien Harris run for 3 yds to the Lvile 14
4 	 3 	 Damien Harris run for 5 yds to the Lvile 9 for a 1ST down
4 	 4 	 Tua Tagovailoa pass incomplete to Jerry Jeudy
4 	 5 	 Tua Tagovailoa run for 9 yds for a TD, (Austin Jones KICK)
5 	 0 	 Joseph Bulovas kickoff for 65 yds for a touchback
5 	 1 	 ALABAMA Penalty, Defensive Pass Interference (Xavier McKinney) to the Lvile 40 for a 1ST down
5 	 2 	 Trey Smith run for 7 yds to the Lvile 47
5 	 3 	 Jawon Pass pass complete to Seth Dawkins for 3 yds to the 50 yard line for a 1ST down
5 	 4 	 Jawon Pass run for a loss of 2 yards to the Lvile 48
5 	 5 	 Jawon Pass pass complete to Trey Smith for 4 yds to the Alab 48
5 	 6 	 Jawon Pass run for 11 yds to the Alab 37 for a 1ST down
5 	 7 	 Jawon Pass pass complete to Dez Fitzpatrick for 13 yds to the Alab 24 for a 1ST down
5 	 8 	 LOUISVILLE Penalty, Substitution Infraction (-5 Yards) to the Alab 29
5 	 9 	 End of 1st Quarter
5 	 10 	 Dae Williams run for 3 yds to the Alab 26
5 	 11 	 Jawon Pass pass intercepted Deionte Thompson return for 25 yds to the Alab 25
6 	 0 	 Najee Harris run for 6 yds to the Alab 31
6 	 1 	 Tua Tagovailoa pass complete to Jerry Jeudy for 11 yds to the Alab 42 for a 1ST down
6 	 2 	 Tua Tagovailoa pass complete to Jaylen Waddle for 49 yds to the Lvile 9 for a 1ST down
6 	 3 	 Tua Tagovailoa run for 8 yds to the Lvile 1
6 	 4 	 Najee Harris run for no gain to the Lvile 1
6 	 5 	 LOUISVILLE Penalty, Substitution Infraction ( Yards) to the Lvile 1
6 	 6 	 Najee Harris run for 1 yd for a TD, (Austin Jones KICK)
7 	 0 	 Joseph Bulovas kickoff for 65 yds , Cornelius Sturghill return for 14 yds 

In [ ]:


            
    def fixDrive(self, gameID, driveNo, drive):
        if gameID == '401012747':
            if driveNo == 27:
                drive['HomeScore'] = ['10']
                
        if gameID == '401012795':
            if driveNo == 11:
                drive['Posession'] = ['12']
                
        if gameID == '401012777':
            if driveNo == 29:
                drive['Posession'] = ['204'] #'"ORST"]
                drive['HomeScore'] = ['41']

        return drive
    
            
    def insertMissingData(self, gameID, driveNo, fullDriveData, debug=False):
        if gameID == "401012295":
            if driveNo == 1:
                fullDriveData.plays[7].play.pa.runback = False                
        if gameID == "401012684":
            if driveNo == 27:
                fullDriveData.possession = "UNC"
                for i in range(len(fullDriveData.plays)):
                    fullDriveData.plays[i].possession = "UNC"
        if gameID == "401012350":
            if driveNo == 21:
                tmp = fullDriveData.plays[3]
                fullDriveData.plays[3] = fullDriveData.plays[4]
                fullDriveData.plays[4] = tmp
                
                ptype = playtype(fullDriveData.plays[3].play.text)
                play  = ptype.getPlay()
                play.analyze(debug=False)
                fullDriveData.plays[3].play = play
                ptype = playtype(fullDriveData.plays[4].play.text)
                play  = ptype.getPlay()
                play.analyze(debug=False)
                fullDriveData.plays[4].play = play
        if gameID == "401012348":
            if driveNo == 7:
                lastplay = fullDriveData.plays[-1]
                start    = lastplay.start
                playText = "Ryley Guay 37 yd FG GOOD"                
                ptype = playtype(playText)
                play  = ptype.getPlay()                
                if play is None:
                    raise ValueError("Could not determine play type from {0}".format(playText))
                play.analyze(debug=False)
                playStart = playstartclass(down=4, togo=start.togo, startY=start.startY, side=start.side)
                playStart.setQuarter(start.quarter)
                playStart.setGameClock(start.gameclock)
                playResults = playclass(possession=fullDriveData.possession, start=playStart, play=play)
                fullDriveData.plays.append(playResults)
            if driveNo == 8:
                lastplay = fullDriveData.plays[0]
                playText = "No Play"
                ptype = playtype(playText)
                play  = ptype.getPlay()
                if play is None:
                    raise ValueError("Could not determine play type from {0}".format(playText))
                play.analyze(debug=False)
                lastplay.play = play                
        if gameID == "401012347":
            if driveNo == 0:
                lastplay = fullDriveData.plays[0]
                playText = lastplay.play.text
                playText = playText.replace("return for 17 yds", "return for 17 yds for a TD")
                ptype = playtype(playText)
                play  = ptype.getPlay()
                if play is None:
                    raise ValueError("Could not determine play type from {0}".format(playText))
                play.analyze(debug=False)
                lastplay.play = play
                lastplay.play.pa.touchdown = True
        if gameID == "401012341":
            if driveNo == 21:
                lastplay = fullDriveData.plays[-1]
                playText = lastplay.play.text
                playText = "{0} (Cooper Garcia Kick)".format(playText)                
                ptype = playtype(playText)
                play  = ptype.getPlay()
                if play is None:
                    raise ValueError("Could not determine play type from {0}".format(playText))
                play.analyze(debug=False)
                lastplay.play = play
        if gameID == "401012316":
            if driveNo == 14:
                lastplay = fullDriveData.plays[-1]
                lastplay.play.pa.fumble = False
                lastplay.play.pa.touchdown = True
        if gameID == "401012302":
            if driveNo == 15:
                lastplay = fullDriveData.plays[-2]
                lastplay.play.pa.fumble = False
                lastplay.play.pa.touchdown = True
        if gameID == "401012289":
            if driveNo == 20:
                lastplay = fullDriveData.plays[-2]
                playText = lastplay.play.text
                playText = playText.replace(" (", " For a TD (")             
                ptype = playtype(playText)
                play  = ptype.getPlay()
                if play is None:
                    raise ValueError("Could not determine play type from {0}".format(playText))
                play.analyze(debug=False)
                lastplay.play = play
                lastplay.play.pa.touchdown = True
        if gameID == "401012263":
            if driveNo == 18:
                lastplay = fullDriveData.plays[-2]
                playText = lastplay.play.text
                playText = "{0} For a TD".format(playText)            
                ptype = playtype(playText)
                play  = ptype.getPlay()
                if play is None:
                    raise ValueError("Could not determine play type from {0}".format(playText))
                play.analyze(debug=False)
                lastplay.play = play
        if gameID == "401012282":
            if driveNo == 8:
                lastplay = fullDriveData.plays[-1]
                start    = lastplay.start
                playText = "Tyrel Dodson 78 Yd Return of Blocked Field Goal (Seth Small Kick)"
                
                ptype = playtype(playText)
                play  = ptype.getPlay()                
                if play is None:
                    raise ValueError("Could not determine play type from {0}".format(playText))
                play.analyze(debug=False)
                play.pa.touchdown = True
                playStart = playstartclass(down=4, togo=start.togo, startY=start.startY, side=start.side)
                playStart.setQuarter(start.quarter)
                playStart.setGameClock(start.gameclock)
                playResults = playclass(possession=fullDriveData.possession, start=playStart, play=play)
                fullDriveData.plays.append(playResults)
                
                headline   = self.parseHeadline(["Touchdown"])
                fullDriveData.headline = headline


        return fullDriveData
            
            
    def augmentPlayWithScore(self, fullDriveData, fieldMap, debug=False):
        if not isinstance(fullDriveData, driveclass):
            raise ValueError("Game result is {0}".format(type(fullDriveData)))

        homeTeam = fieldMap["Home"]
        awayTeam = fieldMap["Away"]
        
        ## Ignore noplay plays
        plays      = [x for x in fullDriveData.plays if x.play.pa.getKey("noplay") is not True]
        headline   = fullDriveData.headline
        possession = fullDriveData.possession

        ## Check For Touchdown
        if isinstance(headline, touchdownplay):
            if sum(play.play.pa.touchdown for play in plays) == 0:
                for i in [(x+1)*-1 for x in list(range(0,len(plays),1))]:
                    if sum([x in plays[i].play.text for x in self.ptc.getPAT()]) > 0:
                        print("Artificially setting play [{0}] to touchdown since text does not reflect it: {1}".format(i, plays[i].play.text))
                        fullDriveData.plays[i].play.pa.touchdown = True
                        break
            
            if sum(play.play.pa.touchdown for play in plays) == 0:
                for i in [-1,-2]:
                    try:
                        lastplay = plays[i]
                    except:
                        continue

                    if lastplay.play.pa.getKey("end") is True:
                        continue

                    if lastplay.play.pa.getKey("penalty") is True or lastplay.play.pa.getKey("penaltyplay") is True:
                        continue

                    if lastplay.play.pa.getKey("fumble") is True:
                        lastplay.play.pa.touchdown = True
                        break
                        if possession == homeTeam:
                            lastplay.possession = awayTeam
                        elif possession == awayTeam:
                            lastpata.possession = homeTeam
                            
                    lastplay.play.pa.touchdown = True
                    print("Artificially setting play [{0}] to touchdown since text does not reflect it".format(i))
                    break
                    
                if sum(play.play.pa.touchdown for play in plays) == 0:
                    for j,play in enumerate(plays):
                        print(j,'\t',play.play.text)
                    raise ValueError("Touchdown still not in text!")


        ## Check For Fumble + Touchdown
        if isinstance(headline, fumbleplay):
            if headline.pa.getKey("touchdown"):
                if sum(play.play.pa.touchdown for play in plays) == 0:
                    print("Artificially setting last play to touchdown since text does not reflect it")
                    lastplay = plays[-1]
                    lastplay.play.pa.touchdown = True
                    if sum(play.play.pa.touchdown for play in plays) == 0:
                        raise ValueError("Touchdown still not in text!")
                
        return fullDriveData
        
        
            
            
            
    def splitChangeOfPossession(self, gameResult, fieldMap, debug=False):
        if not isinstance(gameResult, list):
            raise ValueError("Game result is {0}".format(type(gameResult)))

        homeTeam = fieldMap["Home"]
        awayTeam = fieldMap["Away"]
        
        splitDrives = []
        for idr in range(len(gameResult)):
            driveData  = gameResult[idr]
            if isinstance(driveData, driveclass) is False:
                raise ValueError("Drive Data is not a DriveClass object!")
                
            
            drivePlays = driveData.plays
            if isinstance(drivePlays, list) is False:
                raise ValueError("Drive Plays is not a list object!")


            ######################################################
            ## Split play into [kick] + [return + X]
            ######################################################
            for dp in range(len(drivePlays)):
                playData  = drivePlays[dp]
                if isinstance(playData.play, kickoffplay) and playData.play.pa.getKey("noplay") is not True:
                    if driveData.possession == homeTeam:
                        playData.possession = awayTeam
                    elif driveData.possession == awayTeam:
                        playData.possession = homeTeam
                    newReturnPlay  = returnplay(text=playData.play.text)
                    newReturnPlay.analyze()
                    newReturnPlay.pa.kickoffplay = False
                    newReturnPlay.pa.touchdown = playData.play.pa.touchdown
                    newReturnPlay.pa.fumble    = playData.play.pa.fumble
                    playData.play.pa.touchdown = False
                    playData.play.pa.fumble    = False
                    playData.play.pa.runback   = False
                    newPlayData = playclass(possession=driveData.possession, start=playData.start, play=newReturnPlay)
                    drivePlays.insert(dp+1, newPlayData)
                    break


            ######################################################
            ## Split play into [punt] + [return/safety + X]
            ######################################################
            for dp in range(len(drivePlays)):
                playData  = drivePlays[dp]
                if isinstance(playData.play, puntplay) and playData.play.pa.getKey("noplay") is not True:
                    if playData.play.pa.getKey("safety"):
                        newSafetyPlay  = safetyplay(text=playData.play.text)
                        newSafetyPlay.analyze()
                        newPlayData = playclass(possession=playData.possession, start=playData.start, play=newSafetyPlay)
                        drivePlays.insert(dp+1, newPlayData)
                        break
                    else:
                        playData.play.pa.touchdown = False
                        newReturnPlay  = returnplay(text=playData.play.text)
                        newReturnPlay.pa.punt = True
                        newReturnPlay.analyze()
                        newReturnPlay.pa.punt = False
                        newPlayData = playclass(possession=playData.possession, start=playData.start, play=newReturnPlay)
                        if driveData.possession == homeTeam:
                            newPlayData.possession = awayTeam
                        elif driveData.possession == awayTeam:
                            newPlayData.possession = homeTeam
                        drivePlays.insert(dp+1, newPlayData)
                        break


            ######################################################
            ## Split play into [PAT] + [return + X]
            ######################################################
            for dp in range(len(drivePlays)):
                playData  = drivePlays[dp]
                if dp == len(drivePlays) - 1:
                    if isinstance(playData.play, patplay) and playData.play.pa.getKey("noplay") is not True:
                        if playData.play.pa.getKey("defpat"):
                            playData.play.pa.defpat  = False
                            playData.play.pa.runback = False
                            newReturnPlay  = returnplay(text=playData.play.text)
                            newReturnPlay.analyze()
                            newReturnPlay.pa.defpat = True
                            newReturnPlay.pa.pat    = True
                            newPlayData = playclass(possession=playData.possession, start=playData.start, play=newReturnPlay)
                            if driveData.possession == homeTeam:
                                newPlayData.possession = awayTeam
                            elif driveData.possession == awayTeam:
                                newPlayData.possession = homeTeam
                            drivePlays.insert(dp+1, newPlayData)
                            break


            ######################################################
            ## Split play into [interception] + [return + X]
            ######################################################
            for dp in range(len(drivePlays)):
                playData  = drivePlays[dp]
                if isinstance(playData.play, passingplay) and playData.play.pa.getKey("noplay") is not True:
                    if playData.play.pa.interception is True:
                        if dp == len(drivePlays) - 1:
                            newReturnPlay  = returnplay(text=playData.play.text)
                            newReturnPlay.analyze()
                            newReturnPlay.pa.touchdown = playData.play.pa.touchdown
                            playData.play.pa.touchdown = False
                            newReturnPlay.pa.runback      = playData.play.pa.runback
                            playData.play.pa.runback   = False
                            newPlayData = playclass(possession=playData.possession, start=playData.start, play=newReturnPlay)
                            if playData.possession == homeTeam:
                                newPlayData.possession = awayTeam
                            elif playData.possession == awayTeam:
                                newPlayData.possession = homeTeam
                            drivePlays.insert(dp+1, newPlayData)
                            break


            ######################################################
            ## Split play into [fumble] + [return + X]
            ######################################################
            for dp in range(len(drivePlays)):
                playData  = drivePlays[dp]
                if playData.play.pa.getKey("fumble") and playData.play.pa.getKey("runback"):
                    newReturnPlay  = returnplay(text=playData.play.text)
                    newReturnPlay.analyze()
                    newReturnPlay.pa.fumble = False
                    newReturnPlay.pa.touchdown = playData.play.pa.touchdown
                    playData.play.pa.touchdown = False
                    newPlayData = playclass(possession=playData.possession, start=playData.start, play=newReturnPlay)
                    if playData.possession == homeTeam:
                        newPlayData.possession = awayTeam
                    elif playData.possession == awayTeam:
                        newPlayData.possession = homeTeam
                    drivePlays.insert(dp+1, newPlayData)
                    break


            ######################################################
            ## Split play into [FieldGoal] + [return + X]
            ######################################################
            for dp in range(len(drivePlays)):
                playData  = drivePlays[dp]
                if isinstance(playData.play, fieldgoalplay):
                    if playData.play.pa.getKey("blocked"):
                        newReturnPlay  = returnplay(text=playData.play.text)
                        newReturnPlay.pa.fieldgoal = True
                        newReturnPlay.analyze()
                        newReturnPlay.pa.fieldgoal = False
                        newReturnPlay.pa.touchdown = playData.play.pa.touchdown
                        playData.play.pa.touchdown = False
                        newPlayData = playclass(possession=playData.possession, start=playData.start, play=newReturnPlay)
                        if playData.possession == homeTeam:
                            newPlayData.possession = awayTeam
                        elif playData.possession == awayTeam:
                            newPlayData.possession = homeTeam
                        drivePlays.insert(dp+1, newPlayData)
                        break


            ######################################################
            ## Split play into [fumble] + [return + X] for end of game
            ######################################################
            if idr == len(gameResult) - 1:
                for dp in range(len(drivePlays)):
                    playData  = drivePlays[dp]
                    if playData.play.pa.getKey("fumble") is True and playData.play.pa.getKey("touchdown") is True:
                        playData.play.pa.touchdown = False
                        newReturnPlay  = returnplay(text=playData.play.text)
                        newReturnPlay.analyze()
                        newReturnPlay.pa.fumble = False
                        newReturnPlay.pa.touchdown = True
                        newPlayData = playclass(possession=playData.possession, start=playData.start, play=newReturnPlay)
                        if playData.possession == homeTeam:
                            newPlayData.possession = awayTeam
                        elif playData.possession == awayTeam:
                            newPlayData.possession = homeTeam
                        drivePlays.insert(dp+1, newPlayData)
                        break



            ######################################################
            ## Split play into [safety] + [return + X]
            ######################################################
            for dp in range(len(drivePlays)):
                playData  = drivePlays[dp]
                if isinstance(playData.play, safetyplay):
                    newReturnPlay  = returnplay(text=playData.play.text)
                    newReturnPlay.analyze()
                    newReturnPlay.pa.safety    = False
                    newReturnPlay.pa.safetypts = True
                    newPlayData = playclass(possession=playData.possession, start=playData.start, play=newReturnPlay)
                    if driveData.possession == homeTeam:
                        newPlayData.possession = awayTeam
                    elif driveData.possession == awayTeam:
                        newPlayData.possession = homeTeam
                    drivePlays.insert(dp+1, newPlayData)
                    break
                    
                    
            ######################################################
            ## Split play into [TD] + [PAT + X]
            ######################################################
            for dp in range(len(drivePlays)):
                playData  = drivePlays[dp]
                if playData.play.pa.touchdown is True:
                    if sum([isinstance(x.play, patplay) for x in drivePlays]) == 0:
                        playData.play.pa.addPAT()
                        if playData.play.pa.getKey("addpat") is True:
                            playData.play.pa.addpat = False
                            newPATPlay  = patplay(text=playData.play.text)
                            newPATPlay.analyze()
                            newPATPlay.pa.touchdown = False
                            newPlayData = playclass(possession=playData.possession, start=playData.start, play=newPATPlay)
                            drivePlays.insert(dp+1, newPlayData)
                            break
                            
                                
        return gameResult
            
    
    
    def analyzeGameData(self, gameResult, debug=False):
        if not isinstance(gameResult, list):
            raise ValueError("Game result is {0}".format(type(gameResult)))
            
        print("Drive\tPlay\tPosess\t{0: <5}{1: <5}{2: <10}{3: <3}{4: <4}{5: <5}{6: <5}{7: <30}{8}".format("Gain", "Dist", "Class", "D", "TG", "Side", "Line", "Keys", "Text"))
        for idr,driveData in enumerate(gameResult):
            drivePlays = driveData.plays
            possession = driveData.possession
            headline   = driveData.headline
            detail     = driveData.detail
            print("\n\n")
            print(headline,'\t\t',detail)
            for ip,playData in enumerate(drivePlays):
                playPossession = playData.possession
                print("  {0}\t{1}\t{2}".format(idr,ip,playPossession), end="\t")
            
                keys = ", ".join([k for k,v in playData.play.pa.__dict__.items() if v is True])
                
                    
                try:
                    playResult   = playData.play.result
                    print("{0: <5}".format(playResult), end="")
                except:
                    print("{0: <5}".format(""), end="")
                
                    
                try:
                    playDist   = playData.start.distToEndZone
                    print("{0: <5}".format(playDist), end="")
                except:
                    print("{0: <5}".format(""), end="")
                    
                    
                try:
                    playName   = playData.play.name
                    print("{0: <10}".format(playName), end="")
                except:
                    print("{0: <10}".format(""), end="")
                    
                    
                try:
                    playDown   = playData.start.down
                    print("{0: <3}".format(playDown), end="")
                except:
                    print("{0: <3}".format(""), end="")
                    
                    
                try:
                    playTogo   = playData.start.togo
                    print("{0: <4}".format(playTogo), end="")
                except:
                    print("{0: <4}".format(""), end="")
                    
                    
                try:
                    playSide   = playData.start.side
                    print("{0: <5}".format(playSide), end="")
                except:
                    print("{0: <5}".format(""), end="")
                    
                    
                try:
                    playStartY   = playData.start.startY
                    print("{0: <5}".format(playStartY), end="")
                except:
                    print("{0: <5}".format(""), end="")

                

                print("{0: <30}".format(keys), end="")


                playText   = playData.play.text
                print("{0}".format(playText))



In [ ]:
playText = " x (Austin Jones KICK)"
print(playText[-1])
        pos = 
        print(txt:)


In [ ]:
def siground(x, sig=3):
    if not isinstance(x, (float, int)):
        print("Must pass a real number. You passed a {0}".format(type(x)))
        return None
    
    retval = '%s' % float('%.{0}g'.format(sig) % x)
    
    try:
        result = float(retval)
    except:
        print("Could not turn [{0}] into a float.".format(retval))
        result = None
    
    return result

In [ ]:
x = {}